In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import handytools.navigator as nav
import spectools.models.models as mdl
from handytools.catcher import InputError

1. Find most exciting inputs. (Read paper; for now, can just use most exciting out of the 51*8 shape inputs)
2. Remove one channel, quantify how that impacts the subsequent output. (Quantity: abs difference, *invariance)
3. Can also just look at Frobenius norm of filter matrix.
4. Look at the distribution and decide what to do next.

In [2]:
model = mdl.get_vgg16()
R = nav.npload("/src", "results", f"responses_VGG16", f"key=8_hollow=0_scale=1_light=1_lw=1_preproc=2.npy")

In [4]:
def set_to_zero(model, target, id1, id2):
    flag = True
    for name, param in model.named_parameters():
        if name == target:
            param_clone = param.clone()
            param_clone[id1][id2] = torch.zeros(param_clone[id1][id2].shape)
            param.data = param_clone
            flag = False
    if flag: raise InputError(f"{target} is not found in model.named_parameters().")

In [6]:
set_to_zero(model, "features.11.weight", 435, 0)

In [10]:
params = {name: parameter for name, parameter in model.named_parameters()}
params["features.11.weight"][435][0]

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], grad_fn=<SelectBackward0>)

In [12]:
from copy import deepcopy

model2 = deepcopy(model)

In [13]:
set_to_zero(model2, "features.11.weight", 435, 1)
params1 = {name: parameter for name, parameter in model.named_parameters()}
params2 = {name: parameter for name, parameter in model2.named_parameters()}
print(params1["features.11.weight"][435][1])
print(params2["features.11.weight"][435][1])

tensor([[-0.0118, -0.0092, -0.0111],
        [-0.0070, -0.0173,  0.0005],
        [ 0.0148,  0.0177, -0.0007]], grad_fn=<SelectBackward0>)
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], grad_fn=<SelectBackward0>)


In [14]:
del model2